In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
stroke=pd.read_csv('/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')

In [ ]:
stroke

In [ ]:
stroke.shape

In [ ]:
stroke.info()

In [ ]:
stroke.describe(include=['object'])

In [ ]:
data_cat= [col for col in stroke.columns.values if stroke[col].dtype=='object']

In [ ]:
data_cat

In [ ]:
categorical_values=stroke[data_cat]
categorical_values

In [ ]:
numerical_variables=stroke.drop(categorical_values,axis=1)
numerical_variables

In [ ]:
total=numerical_variables.isnull().sum().sort_values(ascending=True)
percentage=(numerical_variables.isnull().sum())/(numerical_variables.isnull().count()).sort_values(ascending=True)
missing_values_num=pd.concat([total,percentage],axis=1,keys=['total','percenatge'])
missing_values_num

In [ ]:
for col in numerical_variables.columns.values:
    missing_values=stroke[col].isnull().sum()
    
if missing_values>50:
        numerical_variables=numerical_variables.drop(col,axis=1)
else:
        numerical_variables=numerical_variables.fillna(numerical_variables[col].median())


In [ ]:
from scipy.stats import skew
numerical_variables_skew=numerical_variables.apply(lambda x:skew(x.dropna()))

In [ ]:
numerical_variables_skew=numerical_variables_skew[numerical_variables_skew>1]

In [ ]:
numerical_variables_skew

In [ ]:
numerical_variables_skew.index

In [ ]:
numerical_variables[numerical_variables_skew.index]=np.log1p(numerical_variables[numerical_variables_skew.index])

In [ ]:
numerical_variables.hist(figsize=(20,10),bins=50,xlabelsize=8,ylabelsize=8)

In [ ]:
numerical_variables=(numerical_variables-numerical_variables.min())/(numerical_variables.max()-numerical_variables.min())

In [ ]:
numerical_variables

Categorical variables missing values treatment

Checking the number of categorical variables 

In [ ]:
total=categorical_values.isnull().sum().sort_values(ascending=True)
percentage=(categorical_values.isnull().sum())/(categorical_values.isnull().count()).sort_values(ascending=True)
missing_values_cat=pd.concat([total,percentage],axis=1,keys=['total','percenatge'])
missing_values_cat

missing values treatment of categorical variables

In [ ]:
for col in categorical_values.columns.values:
    missing_values=categorical_values[col].isnull().sum()
    
if missing_values>50:
        categorical_values=categorical_values.drop(col,axis=1)
else:
        categorical_values=categorical_values.fillna('XXX')


In [ ]:
new_data=pd.concat([numerical_variables,categorical_values],axis=1)

In [ ]:
new_data.shape

In [ ]:
new_data

In [ ]:
new_data.shape

In [ ]:
new_data

Label Encoder

In [ ]:
from sklearn.preprocessing import LabelEncoder
lab=LabelEncoder()


new_data['gender']=lab.fit_transform(new_data['gender'])
new_data['ever_married']=lab.fit_transform(new_data['ever_married'])
new_data['work_type']=lab.fit_transform(new_data['work_type'])
new_data['Residence_type']=lab.fit_transform(new_data['Residence_type'])
new_data['smoking_status']=lab.fit_transform(new_data['smoking_status'])

splitting OF DATASETS

In [ ]:
x=new_data.iloc[:,new_data.columns!='stroke']
y=new_data.iloc[:,new_data.columns=='stroke']


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=1234)

In [ ]:
from sklearn.preprocessing import StandardScaler
dt=StandardScaler()
x_train_s=dt.fit_transform(x_train)
x_test_s=dt.transform(x_test)

In [ ]:
print('y_train',y_train.shape)
print('y_test',y_test.shape)
print('x_train',x_train.shape)
print('x_test',x_test.shape)

In [ ]:
import statsmodels.api as sm
logit=sm.Logit(y_train,x_train_s)
model1=logit.fit()
model1.summary()

In [ ]:
predict1=model1.predict(x_test_s)

In [ ]:
predicted1=pd.DataFrame(predict1)

In [ ]:
predicted1

In [ ]:
predicted1=np.where(predicted1[0]>0.50,0,1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(predicted1,y_test))


In [ ]:
cols = ['Model','R-Squared Value','ROC Score', 'Precision Score', 'Recall Score','Accuracy Score',
        'Kappa Score']
models_report = pd.DataFrame(columns = cols)
from sklearn import metrics
tmp1 = pd.Series({'Model': " Logistic Regression Base Model",
                  'R-Squared Value': model1.prsquared,
                 'ROC Score' : metrics.roc_auc_score(y_test, predicted1),
                 'Precision Score': metrics.precision_score(y_test, predicted1),
                 'Recall Score': metrics.recall_score(y_test, predicted1),
                 'Accuracy Score': metrics.accuracy_score(y_test, predicted1),
                 'Kappa Score':metrics.cohen_kappa_score(y_test, predicted1)})

model_report1 = models_report.append(tmp1, ignore_index = True)
model_report1